### COCODataset Bounding-Box-Prior

In [2]:
import json
from collections import defaultdict
import numpy as np

In [3]:
annotations = ['E:/yolov3-master/data/annotations/instances_train2014.json','E:/yolov3-master/data/annotations/instances_val2014.json']

In [4]:
# get bbox
bbx = []
for anno in annotations:
    with open(anno, encoding='utf-8') as f:
        data = f.read()
        ad = json.loads(data)
    for image in ad['annotations']:
        _, _, width, height = image['bbox']
        bbx.append([width, height])   

In [5]:
ad.keys()

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])

In [6]:
len(bbx)

896782

In [7]:
bbx[0]

[285.3, 266.03]

In [8]:
bbx = np.array(bbx).reshape(len(bbx), 2)

In [9]:
bbx.shape

(896782, 2)

In [9]:
def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    param:
        boxes: numpy array of shape (r, 2), where r is the number of rows
        k: number of clusters
        dist: distance function
    return:
        numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]  # 9*2, random pick k clusters

    while True:
        for row in range(rows):
            distances[row] = 1 - iou(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

In [ ]:
def avg_iou(boxes, clusters):
    """
    Calculates the average Intersection over Union (IoU) between a numpy array of boxes and k clusters.
    param:
        boxes: numpy array of shape (r, 2), where r is the number of rows
        clusters: numpy array of shape (k, 2) where k is the number of clusters
    return:
        average IoU as a single float
    """
    return np.mean([np.max(iou(boxes[i], clusters)) for i in range(boxes.shape[0])])

In [14]:
def iou(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    param:
        box: tuple or array, shifted to the origin (i. e. width and height)
        clusters: numpy array of shape (k, 2) where k is the number of clusters
    return:
        numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou_ = intersection / (box_area + cluster_area - intersection + 1e-10)

    return iou_

In [ ]:
def get_kmeans(anno, cluster_num=9):

    anchors = kmeans(anno, cluster_num)
    ave_iou = avg_iou(anno, anchors)

    anchors = anchors.astype('int').tolist()

    anchors = sorted(anchors, key=lambda x: x[0] * x[1])

    return anchors, ave_iou

In [ ]:
anchors, ave_iou = get_kmeans(bbx, 9)

> <ipython-input-14-ad5d699d297d>(14)iou()
-> if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
(Pdb) x
array([240.  ,  94.59, 141.83,  22.52,  56.31, 285.3 ,  46.25,  80.  ,
       162.96])
(Pdb) y
array([ 73.71, 127.01,  77.48,  50.84,  41.63, 266.03, 110.6 ,  78.92,
       266.03])
(Pdb) c
> <ipython-input-14-ad5d699d297d>(14)iou()
-> if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
(Pdb) c
> <ipython-input-14-ad5d699d297d>(14)iou()
-> if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
(Pdb) c
> <ipython-input-14-ad5d699d297d>(14)iou()
-> if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
(Pdb) c
> <ipython-input-14-ad5d699d297d>(14)iou()
-> if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
(Pdb) 
